#### Aghsi company
#### noreddine.damane94@gmail.com

# Feature Extraction

## import required libraries

In [5]:
import numpy as np
import math
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import os

In [7]:
## polynômes radiaux de Zernike
def polyradial(r,n,m):
    Rnm = np.zeros(r.shape)
    for s in range(0, ((n-abs(m))//2)+1):
        c = (-1)**(s) * ( math.factorial(n-s) / ((math.factorial(s)) * (math.factorial((n+abs(m))/2-s)) * (math.factorial((n-abs(m))/2-s))))
        Rnm = Rnm + c *  r**(n - 2*s)
        #print("s=",s)
    return Rnm
    

## zernike moments
def Zernikmoment(img,n,m):
    N  = img.shape[0]
    x = np.arange(0 , N)
    x = np.array([x])
    y = x
    X, Y = np.meshgrid(x, y)
    R = np.sqrt((2*X-N+1)**2+(N-1-2*Y)**2) / N
    Theta = np.arctan2((N-1-2*Y),(2*X-N+1))
    R[R>1]= 0 # r inside unit circle
    Rnm = polyradial(R,n,m)
    Product = img*Rnm*np.exp(-1j*m*Theta)
    #Z = (Product).sum()
    Z = (Product.ravel()).sum()
    cnt = np.count_nonzero(R) # count the number of pixels inside the unit circle
    Z = (n+1)*Z/cnt # normalize the amplitude of moments
    A = abs(Z)
    Phi = np.angle(Z)*180/math.pi
    return [Z, float(A), Phi]

## Main()

In [68]:
classes = {
	"circle"   : 1,
	"square"   : 2,
	"star"     : 3,
	"triangle" : 4
}

print('les moments zernike pour n= 1 et m= 5')
n= 5; m= 1
features = []
img_path = "archive/shapes/"
csv_file ="archive/shapes/Zernik_dataset.csv"

for c, idx in classes.items():
    class_folder = img_path + "{}/".format(c)
    for f in os.listdir(class_folder):
        fpath = class_folder + f
        sample = int(f.replace(".png", ""))
        img = cv.imread(fpath, cv.IMREAD_GRAYSCALE)
        img_ = np.logical_not(img)
        img_ = ~img_
        Z_m = Zernikmoment(img_,n,m)
        Z_m = np.array(Z_m)
        Z_m = [idx] + Z_m.reshape((1, 3)).tolist()[0]
        features.append(Z_m)
cols = ["class", "Z", "Amplitude", "Phase"]
df = pd.DataFrame(features, columns=cols)
df.to_csv(csv_file, index=None)
print("Feature Extraction finish!")
df

les moments zernike pour n= 1 et m= 5


C:\Users\HP\AppData\Local\Temp/ipykernel_4280/531996425.py:5: DeprecationWarning: Using factorial() with floats is deprecated
  c = (-1)**(s) * ( math.factorial(n-s) / ((math.factorial(s)) * (math.factorial((n+abs(m))/2-s)) * (math.factorial((n-abs(m))/2-s))))


Feature Extraction finish!


,class,Z,Amplitude,Phase
0,1,0.000985+0.000758j,0.001243+0.000000j,37.583100+0.000000j
1,1,0.001208+0.000928j,0.001523+0.000000j,37.528171+0.000000j
2,1,-0.001986+0.000759j,0.002126+0.000000j,159.092089+0.000000j
3,1,0.001637+0.006489j,0.006693+0.000000j,75.839146+0.000000j
4,1,0.000074-0.000085j,0.000113+0.000000j,-48.868123+0.000000j
...,...,...,...,...
14965,4,-0.086170-0.001971j,0.086193+0.000000j,-178.689417+0.000000j
14966,4,-0.081268+0.006626j,0.081538+0.000000j,175.338804+0.000000j
14967,4,-0.070624+0.018818j,0.073088+0.000000j,165.079691+0.000000j
14968,4,-0.064985+0.030839j,0.071932+0.000000j,154.612802+0.000000j


In [69]:
# standardisation des parametres statistiques:
from sklearn import preprocessing
df = pd.read_csv(csv_file)
X = df["Amplitude"].astype(complex).astype(float)#.reshape(1, -1)
X = [[i, x] for i, x in enumerate(X)]
#std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = X

# target y:
from sklearn import preprocessing 
y = df["class"]

result = {key: {"score" : []} for key, value in classifiers.items()}

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
i = 1
val_score = []

# grille de paramètres
param_grid = {'n_estimators': np.arange(100, 2100, 100) }

# critère de sélection du meilleur modèle
score = 'accuracy'

# cv StratifiedKFold:
cv = StratifiedKFold(5)
# split data train-test


C:\Users\HP\miniconda3\envs\deep\lib\site-packages\pandas\core\dtypes\cast.py:1201: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


In [72]:
sss = StratifiedShuffleSplit(n_splits = 5, test_size=0.3)
for train_index, test_index in sss.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    print(X_train.shape)
X_train

(10479, 2)
(10479, 2)
(10479, 2)
(10479, 2)
(10479, 2)


array([[-0.0790241 , -0.84382667],
       [-1.28324486, -0.88757866],
       [ 0.14011446, -0.41823346],
       ...,
       [ 1.28208785,  1.44206832],
       [-0.06675974, -0.72448187],
       [-0.89009131, -0.88808521]])

In [73]:
X

[[0, 0.0012432433333238015],
 [1, 0.0015227311556031981],
 [2, 0.0021257033830773095],
 [3, 0.006692872753483722],
 [4, 0.00011290162316377803],
 [5, 0.0007850952311322411],
 [6, 0.0005491905815011589],
 [7, 0.0003977938841634189],
 [8, 0.0003429612520680313],
 [9, 0.00026741473575390187],
 [10, 0.0005873545577351645],
 [11, 0.00017806501502443228],
 [12, 0.00035921639261571764],
 [13, 0.0008801957540301871],
 [14, 0.0070042307010041716],
 [15, 0.0005244201799779715],
 [16, 0.00013263602339499365],
 [17, 0.0005890157669678092],
 [18, 0.00042875250547624465],
 [19, 0.0007757533524746693],
 [20, 0.0005080107001696614],
 [21, 0.0001594085398254747],
 [22, 0.0007596315554690592],
 [23, 0.0003963235356507927],
 [24, 0.0011444309790042966],
 [25, 0.0038544772074267787],
 [26, 0.0006072827381375922],
 [27, 0.00015135306941220245],
 [28, 0.00040290355897075795],
 [29, 0.00012220857277946617],
 [30, 0.00041836833664069337],
 [31, 0.00032166607565159796],
 [32, 0.00035458552036754103],
 [33, 0.0